# Data Analysis

## View some data

In [1]:
from constants.data_constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset

id_train_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train",
    name="id_train_dataset"
)
id_test_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation",
    name="id_test_dataset"
)
ood_test_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
    name="maestro_test_dataset"
)

/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    events = get_readable_events(input_ids)
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 30)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic", "vel"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated",
    "vel": "velocity"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df

dataset id_train                                   id_test              ...  \
key         misc onset   dur inst pitch  antic vel    misc onset   dur  ...   
0             AR                                        AR              ...   
1                 4.94  0.65    0    G1  False               0.0   0.5  ...   
2                  0.0  0.15    0    F4  False               0.4  0.35  ...   
3                 0.02  0.28    0    D4  False              0.88  0.25  ...   
4           REST  1.02   0.0             False              1.07  1.64  ...   
5                 5.06  0.61    0    G2  False              1.29  0.96  ...   
6                 5.16  0.33    0    F4  False              1.49  0.12  ...   
7                 5.27   0.6    0   A#4  False              1.61   0.2  ...   
8                 5.43  1.01    0    F5  False              1.68  0.59  ...   
9                 5.62  0.38    0    F4  False              1.76  0.45  ...   
10                5.89  0.74    0    C5  False              1.78  1.02  ...   
11          REST  2.02   0.0             False              2.72  0.29  ...   
12                6.03   0.1    0   A#4  False              2.74  0.14  ...   
13                 6.1  1.07    0    D3  False        REST  3.74   0.0  ...   
14                6.15   0.4    0    F4  False               3.9  0.75  ...   
15                6.26  0.15    0   A#4  False              3.93  0.45  ...   
16                6.35  0.82    0   A#3  False              3.93  0.51  ...   
17                6.52   0.1    0   A#4  False              3.93  0.37  ...   
18                6.58  0.26    0    D4  False              3.93  0.37  ...   
19                6.61  0.07    0    F5  False              4.41  0.21  ...   
20                6.75  0.12    0    F5  False              4.59  0.71  ...   
21                6.84  0.04    0   A#5  False              4.85  0.37  ...   
22                6.93  0.07    0    F6  False              5.29  1.02  ...   
23                6.97  0.07    0   A#5  False              5.81  0.94  ...   
24                6.98  0.46    0    D4  False              5.82  1.25  ...   
25          REST  3.02   0.0             False              6.82  0.19  ...   
26                7.06  0.05    0    F5  False              7.27  0.25  ...   
27                7.19  0.05    0    C5  False              7.47  0.36  ...   
28                7.25  0.47    0    F5  False              7.77  1.24  ...   
29                7.32  0.13    0    C5  False              7.79   0.5  ...   

dataset                  ood_test                                    
key     pitch  antic vel     misc onset   dur inst pitch  antic vel  
0                              AR                                    
1         F#2  False                0.0  0.06    0    C5  False      
2          A3  False               0.09  0.09    0   C#5  False      
3          D4  False               0.18  0.09    0    D5  False      
4         F#3  False               0.19  0.06    0   A#4  False      
5          A3  False               0.27  0.08    0    E5  False      
6          D4  False               0.36  0.11    0    F5  False      
7          E4  False               0.46  0.05    0    D5  False      
8         F#4  False               0.56  0.05    0    G4  False      
9          D4  False               0.64  0.09    0    A4  False      
10         D5  False               0.73  0.08    0   A#4  False      
11         A4  False               0.73  0.19    0    G3  False      
12         A3  False               0.81  0.08    0    C5  False      
13             False                0.9  0.08    0    D5  False      
14         C5  False               0.91  0.17    0   A#3  False      
15         A3  False               0.96  0.06    0   A#4  False      
16         A4  False               1.06  0.07    0    D4  False      
17         F2  False               1.09  0.16    0    B3  False      
18         C3  False               1.18  0.05    0    F4  False      
1

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value 1.0
{'skewness': np.float64(170723.662), 'kurtosis': np.float64(-2.361), 'skewness_p': 0.952, 'kurtosis_p': 0.018, 'royston_p': 0.434, 'hz_p_value': 1.0}


### Extract layers

In [5]:
from extract_layers.extract_layers_main import extract_representations, collate_fn
from constants.model_constants import JORDAN_MODEL_NAME, DEVICE
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(JORDAN_MODEL_NAME).to(DEVICE)
num_layers = 24

id_train_dataloader = DataLoader(
    id_train_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn
)

extract_representations(
    model=model,
    data=id_train_dataloader,
    pooling_function=pooling_function,
    layers=list(range(num_layers + 1)),
)

{0: array([[-1.5858957e-04, -2.2780974e-03, -5.4641580e-03, ...,
          3.6041990e-02,  3.7796646e-02,  3.8871054e-02],
        [ 7.5466487e-05,  6.6216319e-04, -6.6342456e-03, ...,
          3.2185879e-02,  3.5957586e-02,  3.7327852e-02],
        [-1.6285133e-03, -3.2915168e-03, -2.0713529e-03, ...,
          3.7087522e-02,  3.7383165e-02,  3.8296748e-02],
        ...,
        [-1.7962824e-03,  2.3272866e-03, -6.3806227e-03, ...,
          3.4933787e-02,  3.7223965e-02,  3.6779165e-02],
        [ 1.0697738e-03, -6.2378547e-03,  3.5575293e-03, ...,
          3.6529809e-02,  3.8544346e-02,  3.8415130e-02],
        [ 6.0749738e-03, -7.5836191e-03,  1.7723180e-03, ...,
          3.4965035e-02,  3.5934646e-02,  3.8681839e-02]],
       shape=(4550, 2048), dtype=float32),
 1: array([[-0.0273591 ,  0.01079079, -0.05596501, ...,  0.09060913,
          0.09643698,  0.12677993],
        [-0.05586493,  0.00803197, -0.06769733, ...,  0.08572365,
          0.08836669,  0.12163065],
        [-0.0

In [6]:
import numpy as np
import pandas as pd
from constants.file_format import get_extract_layers_file_path
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = get_extract_layers_file_path(
        dataset_name="id_train_dataset",
        pooling_function_name=pooling_function.__name__,
        layer_idx=layer_idx,
    )
    layer_data = np.load(layer_dir)
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing
p_value <0.001


Processing layers:   4%|▍         | 1/25 [00:03<01:15,  3.15s/it]

Processing layer 1
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:06<01:11,  3.10s/it]

...

Processing layer 23
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers:  96%|█████████▌| 24/25 [01:13<00:03,  3.09s/it]

p_value <0.001
Processing layer 24
Layer data shape: (4550, 2048)
Running test mardia
  Subsampling to 4550 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4550 samples and 100 features for testing


Processing layers: 100%|██████████| 25/25 [01:16<00:00,  3.07s/it]

p_value <0.001


In [7]:
all_metrics

,skewness,kurtosis,skewness_p,kurtosis_p,royston_p,hz_p_value
0,885535.187,218.302,0.001,0.001,0.001,0.001
1,931397.042,192.624,0.001,0.001,0.001,0.001
2,1008562.214,139.916,0.001,0.001,0.001,0.001
3,1148154.149,163.006,0.001,0.001,0.001,0.001
4,1141945.628,165.642,0.001,0.001,0.001,0.001
5,1223617.620,187.258,0.001,0.001,0.001,0.001
6,1136145.739,180.864,0.001,0.001,0.001,0.001
7,1156076.546,164.757,0.001,0.001,0.001,0.001
8,1127764.005,160.684,0.001,0.001,0.001,0.001
9,989704.314,106.589,0.001,0.001,0.001,0.001


In [8]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)